02_modeling.ipynb (Pemodelan dan Evaluasi)
Fokus: Melatih algoritma dan mengukur performa.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, 
    accuracy_score, precision_recall_curve, roc_curve
)

# 1. LOAD DATA
print("📂 Loading and Auditing Data...")
df_clean = pd.read_csv('../data/processed/diabetes_cleaned.csv')

# --- TAMBAHAN VISUAL 1: Korelasi Antar Fitur ---
plt.figure(figsize=(10, 8))
sns.heatmap(df_clean.corr(), annot=True, cmap='RdYlGn', fmt='.2f')
plt.title('Korelasi Fitur: Lihat Hubungan Gula Darah & Outcome')
plt.show()

# 2. FEATURE ENGINEERING (Kunci naikin akurasi ke 80%+)
# Kita buat fitur interaksi agar model lebih pintar
df_clean['Glucose_Age_Interaction'] = df_clean['Glucose'] * df_clean['Age']
df_clean['BMI_Glucose_Ratio'] = df_clean['Glucose'] / (df_clean['BMI'] + 1)

X = df_clean.drop('Outcome', axis=1)
y = df_clean['Outcome']

# 3. SPLIT DATA
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. TUNING PIPELINE (Hyper-Focused)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(random_state=42, class_weight='balanced_subsample'))
])

# Parameter yang lebih sempit tapi dalam
param_grid = {
    'model__n_estimators': [200, 300, 500],
    'model__max_depth': [10, 15, 20],
    'model__min_samples_split': [2, 5],
    'model__criterion': ['entropy', 'gini']
}

print("🔍 Tuning Model (Searching for >80% Accuracy)...")
cv = StratifiedKFold(n_splits=5)
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

# 5. EVALUASI & VISUALISASI LANJUTAN
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

print("\n" + "="*50)
print(f"🎯 FINAL TRAINING ACCURACY: {accuracy_score(y_test, y_pred):.2%}")
print(f"📈 FINAL ROC-AUC: {roc_auc_score(y_test, y_proba):.2%}")
print("="*50)

# --- TAMBAHAN VISUAL 2: Distribusi Probabilitas Prediksi ---
# Ini untuk memastikan prediksi tidak "ngumpul" di angka 70% saja
#[Image of a probability distribution plot for binary classification showing separation between classes]
plt.figure(figsize=(10, 5))
sns.histplot(y_proba[y_test == 0], color='blue', label='Sehat (Actual)', kde=True, element="step")
sns.histplot(y_proba[y_test == 1], color='red', label='Diabetes (Actual)', kde=True, element="step")
plt.axvline(0.5, color='black', linestyle='--')
plt.title('Distribusi Probabilitas: Apakah Model Bisa Membedakan Kelas?')
plt.xlabel('Probability of Diabetes')
plt.legend()
plt.show()

# --- TAMBAHAN VISUAL 3: Feature Importance ---
importances = best_model.named_steps['model'].feature_importances_
feat_imp = pd.Series(importances, index=X.columns).sort_values(ascending=True)
plt.figure(figsize=(8, 6))
feat_imp.plot(kind='barh', color='darkgreen')
plt.title('Fitur Paling Berpengaruh (Termasuk Fitur Baru)')
plt.show()

# 6. SIMPAN MODEL
os.makedirs('../models', exist_ok=True)
joblib.dump(best_model, '../models/best_model.pkl')
print("\n✅ Model Gacor v2 disimpan!")

=== Model Evaluation ===
Accuracy: 0.7403

Confusion Matrix:
[[85 15]
 [25 29]]

Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.85      0.81       100
           1       0.66      0.54      0.59        54

    accuracy                           0.74       154
   macro avg       0.72      0.69      0.70       154
weighted avg       0.73      0.74      0.73       154


=== Files Saved Successfully ===
1. preprocessing.pkl saved in ../models/
   - Contains: scaler, feature_names, n_features
2. best_model.pkl saved in ../models/
   - Contains: trained model, accuracy, features metadata

File exists in models directory:
- preprocessing.pkl: True
- best_model.pkl: True
